In [1]:
import numpy as np
import pandas as pd
import scipy.stats as ss
from typing import List

# Data loading

In [2]:
# Load your data
df_tmp = pd.read_parquet("data/empatica/tmp.parquet").set_index("timestamp")
df_gsr = pd.read_parquet("data/empatica/gsr.parquet").set_index("timestamp")
df_acc = pd.read_parquet("data/empatica/acc.parquet").set_index("timestamp")

# !!! The data MUST datetime Indexed
assert isinstance(df_tmp.index, pd.DatetimeIndex)

print(df_tmp.head(3))   # ~4Hz
print('-'*60)
print(df_acc.head(6))  # ~32Hz
print('-'*60)
print(df_gsr.head(3))   # ~4Hz

                                         TMP
timestamp                                   
2017-06-13 14:22:13+02:00         382.209991
2017-06-13 14:22:13.250000+02:00  382.209991
2017-06-13 14:22:13.500000+02:00  382.209991
------------------------------------------------------------
                                  ACC_x  ACC_y  ACC_z
timestamp                                            
2017-06-13 14:22:13+02:00             0      5     63
2017-06-13 14:22:13.031250+02:00      0      5     63
2017-06-13 14:22:13.062500+02:00      0      5     63
2017-06-13 14:22:13.093750+02:00      0      5     63
2017-06-13 14:22:13.125000+02:00      0      5     63
2017-06-13 14:22:13.156250+02:00     -1      5     63
------------------------------------------------------------
                                       EDA
timestamp                                 
2017-06-13 14:22:13+02:00         0.000000
2017-06-13 14:22:13.250000+02:00  0.400309
2017-06-13 14:22:13.500000+02:00  0.475767


**note**: The `ACC` signal is sampled at a different sample frequency.

> So we deal with `multivariate data` of which each modality has a different sample-frequency.

# Series Processing pipeline

In [3]:
try: 
    from tsflex.processing import SeriesProcessor, SeriesPipeline
    from tsflex.processing.logger import get_processor_logs
except:
    import sys
    sys.path.append('../')
    from tsflex.processing import SeriesProcessor, SeriesPipeline
    from tsflex.processing.logger import get_processor_logs

> **note**: processing takes place on one or multiple flat series as is and does not use window-stride parameters<br>the **output** of a processing step should either be `one or multiple` existing **series** (which will be updated in the internal representation ) and/or `one-multiple` new series.

**[link to docs](https://predict-idlab.github.io/tsflex/features/index.html#getting-started)**

![series uml](https://raw.githubusercontent.com/predict-idlab/tsflex/main/docs/_static/series_uml.png)

As shown above, there are 2 relevant classes for processing.

1. [SeriesPipeline](https://predict-idlab.github.io/tsflex/processing/#tsflex.processing.SeriesPipeline): serves as a pipeline, withholding the to-be-applied processing steps
2. [SeriesProcessor](https://predict-idlab.github.io/tsflex/processing/#tsflex.processing.SeriesProcessor): an instance of this class describes a processing step.<br>
   Processors are defined by:<br>
   * `function`: the Callable processing-function - e.g. scipy.signal.detrend
   * `series_names`: the name(s) of the series on which the processing function should be applied
   * `**kwargs`: the keyword arguments for the function.

The snippets below show how the SeriesPipeline & SeriesProcessor interplay:

In [4]:
# --------------------- some custom signal procesisng functions ---------------------
from scipy.signal import savgol_filter

def clip_quantiles(sig: pd.Series, lower_q=0.01, upper_q=0.99) -> np.ndarray:
    # Note that this function induces a data leakage
    quantile_vals = np.quantile(sig, q=[lower_q, upper_q])
    return np.clip(sig, *quantile_vals)

def smv(*sigs) -> pd.Series:
    # creates a new series
    sig_prefixes = set(sig.name.split('_')[0] for sig in sigs)
    result = np.sqrt(np.sum([np.square(sig) for sig in sigs], axis=0))
    return pd.Series(result, index=sigs[0].index, name='|'.join(sig_prefixes)+'_'+'SMV')

In [5]:
# -- 1.  Create the series processors (with their keyword arguments)
clipper_tmp = SeriesProcessor(clip_quantiles, series_names="TMP", lower_q=0, upper_q=0.999)
savgol_eda = SeriesProcessor(savgol_filter, "EDA", window_length=5, polyorder=2)
savgol_acc = SeriesProcessor(savgol_filter, ["ACC_x", "ACC_y", "ACC_z"], window_length=33, polyorder=2)
smv_processor = SeriesProcessor(smv, ("ACC_x", "ACC_y", "ACC_z"))
clipper_smv = SeriesProcessor(clip_quantiles, "ACC_SMV")

# -- 2. Create the series pipeline which wittholds a list of processors
series_pipe = SeriesPipeline(
    processors=[clipper_tmp, savgol_eda, savgol_acc, smv_processor, clipper_smv]
)
series_pipe

[
	clip_quantiles {'lower_q': 0, 'upper_q': 0.999} :  ('TMP',)
	savgol_filter {'window_length': 5, 'polyorder': 2} :  ('EDA',)
	savgol_filter {'window_length': 33, 'polyorder': 2} :  ('ACC_x',) ('ACC_y',) ('ACC_z',)
	smv {} :  ('ACC_x', 'ACC_y', 'ACC_z')
	clip_quantiles {} :  ('ACC_SMV',)
]

In [6]:
# 3. Process the data
out_data : List[pd.Series] = series_pipe.process(
    [df_gsr, df_acc, df_tmp]
)

# update the df_gsr and the df_tmp, and save the df_smv
for series in out_data:
    print(series.name)

EDA
ACC_x
ACC_y
ACC_z
TMP
ACC_SMV


## Processing function logging

The `ProcessingPipeline` can also log the duration of it's `SeriesProcessor` steps

To do so, one needs to set the `logging_file_path` argumetn and then call the `get_processor_logs()` method with this corresponding path to see the output dataframe.

In [7]:
_ = series_pipe.process([df_gsr, df_acc, df_tmp], logging_file_path="example_processing_logs.log")

In [8]:
get_processor_logs("example_processing_logs.log")

,log_time,function,series_names,output_names,duration,duration %
0,2023-05-05 12:16:53.596,clip_quantiles,"(TMP,)",TMP,0 days 00:00:00.002578633,4.73
1,2023-05-05 12:16:53.597,savgol_filter,"(EDA,)",EDA,0 days 00:00:00.000956786,1.76
2,2023-05-05 12:16:53.628,savgol_filter,"(ACC_x,), (ACC_y,), (ACC_z,)","ACC_x, ACC_y, ACC_z",0 days 00:00:00.030402122,55.82
3,2023-05-05 12:16:53.635,smv,"(ACC_x, ACC_y, ACC_z)",ACC_SMV,0 days 00:00:00.006881422,12.63
4,2023-05-05 12:16:53.649,clip_quantiles,"(ACC_SMV,)",ACC_SMV,0 days 00:00:00.013646172,25.05


This is especially useful to see which of the processing step is the bottleneck

# Feature extraction

In [9]:
try: 
    from tsflex.features import FeatureCollection, FuncWrapper
    from tsflex.features import FeatureDescriptor, MultipleFeatureDescriptors
    from tsflex.features.logger import get_feature_logs
except:
    import sys
    sys.path.append('../')
    from tsflex.features import FeatureCollection, FuncWrapper
    from tsflex.features import FeatureDescriptor, MultipleFeatureDescriptors
    from tsflex.features.logger import get_feature_logs

/home/jeroen/.cache/pypoetry/virtualenvs/tsflex-5Y4iXlk8-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Defining functions

**[link to docs](https://predict-idlab.github.io/tsflex/features/index.html#getting-started)**

![features uml](https://raw.githubusercontent.com/predict-idlab/tsflex/main/docs/_static/features_uml.png)

As shown above, there are 3 relevant classes for feature-extraction.

1. [FeatureCollection](https://predict-idlab.github.io/tsflex/features/#tsflex.features.FeatureCollection): serves as a registry, withholding the to-be-calculated _features_
2. [FeatureDescriptor](https://predict-idlab.github.io/tsflex/features/#tsflex.features.FeatureDescriptor): an instance of this class describes a _feature_. <br>Features are defined by:
      * `series_name`: the names of the signal(s) which this feature will use. 
      * `function`: the _Callable_ feature-function - e.g. _np.mean_
      * `window`: the _time-based_ window -  e.g. _"1hour"_
      * `stride`: the _time-based_ stride - e.g. _"2days"_
3. [NumpyFuncWrapper](https://predict-idlab.github.io/tsflex/features/#tsflex.features.NumpyFuncWrapper): a wrapper around _Callable_ functions, intended for advanced feature function definitions, such as:
    * features with multiple output columns
    * passing _**kwargs_ to feature functions


**Note**: this library does `not` provide any feature-functions as:
* There already exist many other feature extraction libraries such as numpy, scipy, tsfresh with which `tsflex` integrates.
* (Relevant) features are dependent on the objective and signals-modalites, making features methods very problem specific.
* Finally, as can be seen in the example below, our `NumpyFuncWrapper`'s `func`-attribute is versatile enough to wrap the end-user's desired features.

In [10]:
# --------------------- some custom feature extraction functions ---------------------
# -- 1. one-to-many functions
#    To compute quantiles, you need sort the windowed data, which is a rather expensive
#    operation O(n*log(n)). Hence, you might want to calculate all your desired 
#    quantiles in a single function-wrapper, returning multiple outputs.

quantiles = [0.25, 0.5, 0.75]
f_quantiles = FuncWrapper(
    func=np.quantile,  # the wrapped function that will operate on numpy arrays
    output_names=[f"quantile_{q}" for q in quantiles],  # the output column names
    q=quantiles,  # optional - additional function-related kwargs
)


# -- 2. in-line functions
#    You can define your functions locally; these will serialize flawlessly
def slope(x):
    return np.polyfit(np.arange(0, len(x)), x, 1)[0]

f_slope = FuncWrapper(slope, output_names="slope")

# -- 3. Lambda's
#    Or even use lambda's and other modules' functions
f_rms = FuncWrapper(lambda x: np.sqrt(np.mean(x ** 2)), output_names="rms")
f_area = FuncWrapper(np.sum, output_names="area")


# (For convenience) we store the constructed `NumpyFuncWrappers` in a list
segment_funcs = [
    np.mean,
    np.std,
    np.var,
    np.max,
    np.min,
    ss.skew,  # use other libraries such as scipy
    ss.kurtosis,
    f_quantiles,
    f_slope,
    f_rms,
    f_area,
]
segment_funcs

[<function numpy.mean(a, axis=None, dtype=None, out=None, keepdims=<no value>, *, where=<no value>)>,
 <function numpy.std(a, axis=None, dtype=None, out=None, ddof=0, keepdims=<no value>, *, where=<no value>)>,
 <function numpy.var(a, axis=None, dtype=None, out=None, ddof=0, keepdims=<no value>, *, where=<no value>)>,
 <function numpy.amax(a, axis=None, out=None, keepdims=<no value>, initial=<no value>, where=<no value>)>,
 <function numpy.amin(a, axis=None, out=None, keepdims=<no value>, initial=<no value>, where=<no value>)>,
 <function scipy.stats._stats_py.skew(a, axis=0, bias=True, nan_policy='propagate', *, keepdims=False)>,
 <function scipy.stats._stats_py.kurtosis(a, axis=0, fisher=True, bias=True, nan_policy='propagate', *, keepdims=False)>,
 FuncWrapper(quantile, ['quantile_0.25', 'quantile_0.5', 'quantile_0.75'], {'q': [0.25, 0.5, 0.75]}),
 FuncWrapper(slope, ['slope'], {}),
 FuncWrapper(<lambda>, ['rms'], {}),
 FuncWrapper(sum, ['area'], {})]

## Single series feature extraction

The defined functions above will be encapsulated in a [FeatureDescriptor](https://predict-idlab.github.io/tsflex/features/index.html#tsflex.features.FeatureDescriptor) object.

A `FeatureDescriptor` describes a feature, and has 4 main attributes:

<center>

## Featuredescriptor constructor args:

| attribute | type | info |
| --------: | :--: | ---- |
| `function` | Union[Callable, <br>NumpyFuncWrapper] | The `function` that calculates this feature. |
|      `key` |           Tuple[str, ...]             | The signal key; i.e., the `pd.DataFrame` column name or <br> `pd.Series` name on which the function will operate. |
|   `window` |      Union[str, pd.timedelta]         | The window size on which this feature will be applied, <br> expressed in a **time-based** manner. |
||
    
|  attribute 	|                  type                 	| info                                                                                                             	|
|-----------:	|:-------------------------------------:	|------------------------------------------------------------------------------------------------------------------	|
| `function` 	| Union[Callable, <br>NumpyFuncWrapper] 	| The `function` that calculates this feature.                                                                     	|
|      `key` 	|                 Tuple[str, ...]                	| The signal key; i.e., the `pd.DataFrame` column name or <br> `pd.Series` name on which the function will operate.     	|
|   `window` 	|                  Union[str, pd.timedelta]               	| The window size on which this feature will be applied, <br> expressed in a **time-based** manner. 	|
|   `stride` 	|                  Union[str, pd.timedelta]                  	| The stride of the window rolling process, also as <br> expressed in a **time-based** manner|

</center>


**note**: [MultipleFeatureDescriptor](https://predict-idlab.github.io/tsflex/features/index.html#tsflex.features.MultipleFeatureDescriptors) is actaully a factory for `FeatureDescriptor` objects.

### Fixed window size & stride

**note**: this functionality is exposed by most existing time-series libraries (often on a sample-based matter).

In this example, we will use the _temperature_ signal from a wearable

In [11]:
df_tmp.sample(2)

,TMP
timestamp,
2017-06-13 15:41:21+02:00,34.369999
2017-06-13 15:00:18.750000+02:00,35.500000


Note how the `TMP`-column is used as signal_key in the `FeatureCollection`

In [12]:
# Define the sample frequency and window size
tmp_feat_extr = FeatureCollection(
    feature_descriptors=[
        MultipleFeatureDescriptors(
            functions=segment_funcs,  # The list of functions we constructed earlier
            series_names=["TMP"],
            windows='60s',
            strides='30s',
        )
    ]
)

# The FeatureCollection's __repr__() gives a nice overview of the structure
tmp_feat_extr

TMP: (
	win: 1m    : [
		FeatureDescriptor - func: FuncWrapper(mean, ['mean'], {})    stride: ['30s'],
		FeatureDescriptor - func: FuncWrapper(std, ['std'], {})    stride: ['30s'],
		FeatureDescriptor - func: FuncWrapper(var, ['var'], {})    stride: ['30s'],
		FeatureDescriptor - func: FuncWrapper(amax, ['amax'], {})    stride: ['30s'],
		FeatureDescriptor - func: FuncWrapper(amin, ['amin'], {})    stride: ['30s'],
		FeatureDescriptor - func: FuncWrapper(skew, ['skew'], {})    stride: ['30s'],
		FeatureDescriptor - func: FuncWrapper(kurtosis, ['kurtosis'], {})    stride: ['30s'],
		FeatureDescriptor - func: FuncWrapper(quantile, ['quantile_0.25', 'quantile_0.5', 'quantile_0.75'], {'q': [0.25, 0.5, 0.75]})    stride: ['30s'],
		FeatureDescriptor - func: FuncWrapper(slope, ['slope'], {})    stride: ['30s'],
		FeatureDescriptor - func: FuncWrapper(<lambda>, ['rms'], {})    stride: ['30s'],
		FeatureDescriptor - func: FuncWrapper(sum, ['area'], {})    stride: ['30s'],
	]
)

In [13]:
# to extract the features we just call the collection's `calculate()` function
extracted_feats = tmp_feat_extr.calculate(
    data=out_data,    # The signals on which features are calculated
    # NOTE: out_data withholds the updated TMP signal
    return_df=True,  # If true, an outer merge on the feature-outputs will be performed
    n_jobs=2         # If > 1, the feature extraction is parallellized
)

extracted_feats.sample(2)

,TMP__amax__w=1m,TMP__amin__w=1m,TMP__area__w=1m,TMP__kurtosis__w=1m,TMP__mean__w=1m,TMP__quantile_0.25__w=1m,TMP__quantile_0.5__w=1m,TMP__quantile_0.75__w=1m,TMP__rms__w=1m,TMP__skew__w=1m,TMP__slope__w=1m,TMP__std__w=1m,TMP__var__w=1m
timestamp,,,,,,,,,,,,,
2017-06-13 15:19:13+02:00,35.029999,34.930000,8394.200195,-0.810092,34.975834,34.950001,34.990002,34.990002,34.975845,-0.058798,-0.000345,0.028884,0.000834
2017-06-13 16:17:43+02:00,30.250000,29.809999,7206.559570,-1.437857,30.027332,29.889999,30.030001,30.170000,30.027679,0.071025,0.001219,0.144382,0.020846


### Multiple `time-based` window sizes and strides

_In this example, we use **multiple** stride-window-size combinations on a wearables' ElectorDermal Activity (EDA)_

Note that we do not use int-based window-stride combinations, but `time-based` ones. Also take a closer look at the `__repr__` string.

In [14]:
# PoC: we will select a random combination of the window_size stride combination
window_size_s = ['30s', '120s', '90s', '1h']
stride_size_s = ['15s', '30s']

import random

gsr_feat_extr = FeatureCollection(
    [
        FeatureDescriptor(
            series_name="EDA",
            window=random.choice(window_size_s),
            stride=random.choice(stride_size_s),
            function=f,
        )
        for f in segment_funcs
    ]
)

# the __repr__ string outputs the windows & strides in a time-string representation :)
print(gsr_feat_extr)
print('-'*60)
gsr_feat_extr.calculate(df_gsr, return_df=True, show_progress=False, n_jobs=None).sample(2)

EDA: (
	win: 1h    : [
		FeatureDescriptor - func: FuncWrapper(mean, ['mean'], {})    stride: ['30s'],
		FeatureDescriptor - func: FuncWrapper(skew, ['skew'], {})    stride: ['15s'],
		FeatureDescriptor - func: FuncWrapper(kurtosis, ['kurtosis'], {})    stride: ['30s'],
		FeatureDescriptor - func: FuncWrapper(sum, ['area'], {})    stride: ['30s'],
	]
	win: 30s   : [
		FeatureDescriptor - func: FuncWrapper(std, ['std'], {})    stride: ['15s'],
		FeatureDescriptor - func: FuncWrapper(var, ['var'], {})    stride: ['15s'],
		FeatureDescriptor - func: FuncWrapper(quantile, ['quantile_0.25', 'quantile_0.5', 'quantile_0.75'], {'q': [0.25, 0.5, 0.75]})    stride: ['30s'],
	]
	win: 1m30s : [
		FeatureDescriptor - func: FuncWrapper(amax, ['amax'], {})    stride: ['15s'],
		FeatureDescriptor - func: FuncWrapper(amin, ['amin'], {})    stride: ['15s'],
		FeatureDescriptor - func: FuncWrapper(<lambda>, ['rms'], {})    stride: ['30s'],
	]
	win: 2m    : [
		FeatureDescriptor - func: FuncWrapper(slope,

,EDA__amax__w=1m30s,EDA__amin__w=1m30s,EDA__area__w=1h,EDA__kurtosis__w=1h,EDA__mean__w=1h,EDA__quantile_0.25__w=30s,EDA__quantile_0.5__w=30s,EDA__quantile_0.75__w=30s,EDA__rms__w=1m30s,EDA__skew__w=1h,EDA__slope__w=2m,EDA__std__w=30s,EDA__var__w=30s
timestamp,,,,,,,,,,,,,
2017-06-13 16:09:43+02:00,1.207757,1.139973,30162.832031,-0.272705,2.094641,1.157878,1.160436,1.166831,1.168666,0.94309,-0.000009,0.005837,0.000034
2017-06-13 15:02:58+02:00,0.780491,0.703754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000113,0.005096,0.000026


**note**: The `NaN` values in the above pd.DataFrame are cause by the outer merge which we do to retain the time-indices. Various feature were extracted at different windows & strides, thus making few features share the same time-indices.
<br><br>
If we set `return_df=False`, a `List[pd.Series]` will be returned

In [15]:
feat_list : List[pd.Series] = gsr_feat_extr.calculate(df_gsr, return_df=False, show_progress=False, n_jobs=None)
print(len(feat_list))
feat_list[0].sample(3)

11


,EDA__area__w=1h
timestamp,
2017-06-13 15:26:43+02:00,15623.870117
2017-06-13 16:18:13+02:00,28911.509766
2017-06-13 16:19:43+02:00,28819.720703


## Multiple series feature extraction

In [16]:
# Construct the feature FeatureCollection
#   =  higher order wrapper which aggregates the featuredescriptions
multimodal_feature_extraction = FeatureCollection(
    feature_descriptors=[gsr_feat_extr, tmp_feat_extr]
)

print(multimodal_feature_extraction)
print('-'*60)

df_feat = multimodal_feature_extraction.calculate(
    [df_gsr, df_tmp], return_df=True
)
df_feat.sample(2)

EDA: (
	win: 1h    : [
		FeatureDescriptor - func: FuncWrapper(mean, ['mean'], {})    stride: ['30s'],
		FeatureDescriptor - func: FuncWrapper(skew, ['skew'], {})    stride: ['15s'],
		FeatureDescriptor - func: FuncWrapper(kurtosis, ['kurtosis'], {})    stride: ['30s'],
		FeatureDescriptor - func: FuncWrapper(sum, ['area'], {})    stride: ['30s'],
	]
	win: 30s   : [
		FeatureDescriptor - func: FuncWrapper(std, ['std'], {})    stride: ['15s'],
		FeatureDescriptor - func: FuncWrapper(var, ['var'], {})    stride: ['15s'],
		FeatureDescriptor - func: FuncWrapper(quantile, ['quantile_0.25', 'quantile_0.5', 'quantile_0.75'], {'q': [0.25, 0.5, 0.75]})    stride: ['30s'],
	]
	win: 1m30s : [
		FeatureDescriptor - func: FuncWrapper(amax, ['amax'], {})    stride: ['15s'],
		FeatureDescriptor - func: FuncWrapper(amin, ['amin'], {})    stride: ['15s'],
		FeatureDescriptor - func: FuncWrapper(<lambda>, ['rms'], {})    stride: ['30s'],
	]
	win: 2m    : [
		FeatureDescriptor - func: FuncWrapper(slope,

,EDA__amax__w=1m30s,EDA__amin__w=1m30s,EDA__area__w=1h,EDA__kurtosis__w=1h,EDA__mean__w=1h,EDA__quantile_0.25__w=30s,EDA__quantile_0.5__w=30s,EDA__quantile_0.75__w=30s,EDA__rms__w=1m30s,EDA__skew__w=1h,...,TMP__kurtosis__w=1m,TMP__mean__w=1m,TMP__quantile_0.25__w=1m,TMP__quantile_0.5__w=1m,TMP__quantile_0.75__w=1m,TMP__rms__w=1m,TMP__skew__w=1m,TMP__slope__w=1m,TMP__std__w=1m,TMP__var__w=1m
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-06-13 15:19:13+02:00,1.272983,1.141252,NaN,NaN,NaN,1.150205,1.156600,1.164273,1.191974,NaN,...,-0.810092,34.975834,34.950001,34.990002,34.990002,34.975845,-0.058798,-0.000345,0.028884,0.000834
2017-06-13 15:29:43+02:00,1.610625,1.311352,16156.117188,-0.311375,1.121953,1.327978,1.352278,1.368264,1.439992,0.933501,...,-0.514506,33.743000,33.730000,33.740000,33.750000,33.743008,0.442661,0.000050,0.022753,0.000518


## Logging

The `FeatureCollection` can also log the duration of it's `Features`

To do so, one needs to set the `logging_file_path` argumetn and then call the `get_feature_logs()` method with this corresponding path to see the output dataframe.

In [17]:
multimodal_feature_extraction.calculate(out_data, logging_file_path="example_feature_logs.log");

In [18]:
get_feature_logs("example_feature_logs.log")

,log_time,function,series_names,window,stride,output_names,duration,duration %
0,2023-05-05 12:17:13.103,sum,"(EDA,)",1h,"(30s,)",EDA__area__w=1h,0 days 00:00:00.003769857,0.21
1,2023-05-05 12:17:13.106,mean,"(EDA,)",1h,"(30s,)",EDA__mean__w=1h,0 days 00:00:00.006607244,0.37
2,2023-05-05 12:17:13.144,var,"(EDA,)",30s,"(15s,)",EDA__var__w=30s,0 days 00:00:00.032368199,1.82
3,2023-05-05 12:17:13.150,amax,"(EDA,)",1m30s,"(15s,)",EDA__amax__w=1m30s,0 days 00:00:00.010895506,0.61
4,2023-05-05 12:17:13.153,amin,"(EDA,)",1m30s,"(15s,)",EDA__amin__w=1m30s,0 days 00:00:00.007409622,0.42
5,2023-05-05 12:17:13.158,std,"(EDA,)",30s,"(15s,)",EDA__std__w=30s,0 days 00:00:00.058665547,3.30
6,2023-05-05 12:17:13.160,<lambda>,"(EDA,)",1m30s,"(30s,)",EDA__rms__w=1m30s,0 days 00:00:00.014571842,0.82
7,2023-05-05 12:17:13.173,mean,"(TMP,)",1m,"(30s,)",TMP__mean__w=1m,0 days 00:00:00.009052561,0.51
8,2023-05-05 12:17:13.177,quantile,"(EDA,)",30s,"(30s,)","EDA__quantile_0.25__w=30s, EDA__quantile_0.5__...",0 days 00:00:00.044969643,2.53
9,2023-05-05 12:17:13.186,amax,"(TMP,)",1m,"(30s,)",TMP__amax__w=1m,0 days 00:00:00.003757103,0.21


This is especially  useful for optimizing to see which features are the bottleneck

## Use case: batch-based feature extraction

In [19]:
from tsflex.chunking import chunk_data

* maybe execute this on a highdimensional series, like the `sleep data`

In [20]:
same_range_chunks = chunk_data(
    data=[df_tmp],
    fs_dict={"EDA": 4, "TMP": 4},
    max_chunk_dur='10min'
)

## Serialization

Serialization is mandatory to store and share your pipelines.
`TODO`

In [21]:
multimodal_feature_extraction.serialize("data/example_serialization.pkl")

# Bonus - Get LAYD: Look At Your Data

And as a bonus, for running/reading this notebook, you get some nice visualization code, for
ofcourse time-series.

In [24]:
# !pip install ipywidgets
# !pip install plotly
import ipywidgets as widgets
import plotly.graph_objects as go
from ipywidgets import interact_manual
from plotly.subplots import make_subplots

In [25]:
df_dict = {"tmp": df_tmp, "gsr": df_gsr}
feat_widget = widgets.SelectMultiple(options=df_feat.columns)
sig_widget = widgets.SelectMultiple(options=["gsr", "tmp"])


@interact_manual
def visuzalize(features=feat_widget, signals=sig_widget):
    row_titles = list(signals) + ["features"] if len(features) else []
    fig = make_subplots(
        rows=len(row_titles),
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.1 / len(row_titles),
        row_titles=row_titles,
    )
    fig.update_layout(height=300 * len(row_titles))

    # first, visualize the "raw" signals
    row_idx = 1
    for sig in signals:
        df_sig = df_dict[sig][10:].resample("1s").mean()
        for col in set(df_sig.columns).difference(["index", "timestamp"]):
            fig.add_trace(
                go.Scattergl(x=df_sig.index, y=df_sig[col], name=col, hoverinfo="skip"),
                row=row_idx,
                col=1,
            )
        row_idx += 1

    # then visualize the features
    for feature in features:
        df_ff = df_feat[feature].dropna()
        fig.add_trace(
            go.Scattergl(
                connectgaps=True,
                x=df_ff.index,
                y=df_ff,
                name=feature,
                hoverinfo="skip",
                mode="markers",
                showlegend=True,
            ),
            row=row_idx,
            col=1,
        )

    return fig.show()

interactive(children=(SelectMultiple(description='features', options=('EDA__amax__w=1m30s', 'EDA__amin__w=1m30…